Analisis exploratorio de datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual
sns.set(style="whitegrid")

# Cargar el archivo CSV
df = pd.read_csv('bank.csv', sep=';')

# Ver las primeras filas
display(df.head())

# Información general del dataset
print("Información general:")
df.info()

# Estadísticas descriptivas para variables numéricas
print("\nEstadísticas descriptivas:")
display(df.describe())

Analizar la variable objetivo : 'y'

In [ ]:
# Distribución de la variable objetivo
print("Distribución de la variable 'y':")
print(df['y'].value_counts(normalize=True))  # proporciones

# Gráfico de distribución
sns.countplot(x='y', data=df)
plt.title('Distribución de la variable objetivo (¿Contrato el depósito?)')
plt.show()



Analisis de valores nulos y unknown

In [ ]:
# Análisis combinado de valores nulos y unknown
print("Análisis de valores nulos y unknown por columna:")

# Crear un DataFrame para el análisis
null_analysis = pd.DataFrame({
    'Valores_Nulos': df.isnull().sum(),
    'Valores_Unknown': (df == 'unknown').sum(),
    'Total_Registros': len(df)
})

# Calcular porcentajes
null_analysis['Porcentaje_Nulos'] = (null_analysis['Valores_Nulos'] / null_analysis['Total_Registros']) * 100
null_analysis['Porcentaje_Unknown'] = (null_analysis['Valores_Unknown'] / null_analysis['Total_Registros']) * 100

# Mostrar solo las columnas que tienen valores nulos o unknown
null_analysis = null_analysis[(null_analysis['Valores_Nulos'] > 0) | (null_analysis['Valores_Unknown'] > 0)]
print("\nResumen de valores nulos y unknown:")
print(null_analysis)

# Visualización
plt.figure(figsize=(12, 6))
null_analysis[['Porcentaje_Nulos', 'Porcentaje_Unknown']].plot(kind='bar')
plt.title('Porcentaje de Valores Nulos y Unknown por Columna')
plt.xlabel('Columnas')
plt.ylabel('Porcentaje')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Análisis detallado de las columnas con valores unknown
print("\nAnálisis detallado de valores 'unknown' por columna:")
for column in df.columns:
    if (df[column] == 'unknown').any():
        print(f"\nColumna: {column}")
        print("Distribución de valores:")
        print(df[column].value_counts(normalize=True).round(3))

Determinar los atributos mas importantes con respecto a la clase objetivo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Configuración visual
sns.set(style="whitegrid")

# Cargar el archivo CSV
df_visualizar = pd.read_csv('bank.csv', sep=';')

# Preparación de datos para el análisis
# Convertir la variable objetivo a numérica
le = LabelEncoder()
df_visualizar['y'] = le.fit_transform(df_visualizar['y'])

# Convertir variables categóricas a numéricas
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df_visualizar[col] = le.fit_transform(df_visualizar[col])

# 1. Análisis de correlación
plt.figure(figsize=(12, 8))
correlation_matrix = df_visualizar.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.show()

# 2. Análisis de importancia de características usando Random Forest
X = df_visualizar.drop('y', axis=1)
y = df_visualizar['y']

# Entrenar un modelo Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

# Obtener importancia de características
feature_importance = pd.DataFrame({
    'Característica': X.columns,
    'Importancia': rf.feature_importances_
})
feature_importance = feature_importance.sort_values('Importancia', ascending=False)

# Visualizar importancia de características
plt.figure(figsize=(12, 6))
sns.barplot(x='Importancia', y='Característica', data=feature_importance)
plt.title('Importancia de Características (Random Forest)')
plt.tight_layout()
plt.show()

# Mostrar las características más importantes
print("\nTop 10 características más importantes:")
print(feature_importance.head(10))

Convertir los unknown a nulos

In [ ]:
# Primero, veamos cuántos valores 'unknown' hay antes de la limpieza
print("Antes de la limpieza:")
print(df.isin(['unknown']).sum())

# Crear una copia del DataFrame
df_clean = df.copy()

# Reemplazar 'unknown' con NaN
df_clean = df_clean.replace('unknown', pd.NA)

# Verificar si quedaron valores 'unknown'
print("\nDespués de la limpieza:")
print(df_clean.isin(['unknown']).sum())

# Mostrar el número de filas antes y después
print(f"\nNúmero de filas antes: {len(df)}")
print(f"Número de filas después: {len(df_clean)}")

# Mostrar las columnas que aún tienen valores 'unknown' si las hay
unknown_columns = df_clean.columns[df_clean.isin(['unknown']).any()].tolist()
if unknown_columns:
    print("\nColumnas que aún tienen valores 'unknown':")
    print(unknown_columns)

In [ ]:
# Análisis combinado de valores nulos y unknown
print("Análisis de valores nulos y unknown por columna:")

# Crear un DataFrame para el análisis
null_analysis = pd.DataFrame({
    'Valores_Nulos': df_clean.isnull().sum(),
    'Valores_Unknown': (df_clean == 'unknown').sum(),
    'Total_Registros': len(df_clean)
})

# Calcular porcentajes
null_analysis['Porcentaje_Nulos'] = (null_analysis['Valores_Nulos'] / null_analysis['Total_Registros']) * 100
null_analysis['Porcentaje_Unknown'] = (null_analysis['Valores_Unknown'] / null_analysis['Total_Registros']) * 100

# Mostrar solo las columnas que tienen valores nulos o unknown
null_analysis = null_analysis[(null_analysis['Valores_Nulos'] > 0) | (null_analysis['Valores_Unknown'] > 0)]
print("\nResumen de valores nulos y unknown:")
print(null_analysis)

# Análisis detallado de las columnas con valores nulos (que eran unknown)
print("\nAnálisis detallado de valores nulos por columna:")
for column in df_clean.columns:
    if df_clean[column].isnull().any():
        print(f"\nColumna: {column}")
        print("Distribución de valores:")
        print(df_clean[column].value_counts(normalize=True).round(3))

Análisis de valores nulos por columna

In [ ]:
print("Porcentaje de valores nulos por columna:")
for col in ['job', 'education', 'contact', 'poutcome']:
    nulos = df_clean[col].isnull().sum()
    porcentaje = (nulos / len(df_clean)) * 100
    print(f"{col}: {porcentaje:.2f}%")

Sustituir valores nulos en job, education y contact por la moda y eliminar las filas de poutcome 

In [ ]:
# Primero, veamos el número de filas original
print(f"Número de filas original: {len(df)}")


# Sustituir valores nulos en job, education y contact por la moda
df_clean['job'] = df_clean['job'].fillna(df_clean['job'].mode()[0])
df_clean['education'] = df_clean['education'].fillna(df_clean['education'].mode()[0])
df_clean['contact'] = df_clean['contact'].fillna(df_clean['contact'].mode()[0])

# Eliminar filas donde poutcome es nulo
df_clean = df_clean.dropna(subset=['poutcome'])

# Verificar resultados
print("\nValores nulos después del tratamiento:")
print(df_clean.isnull().sum())
print(f"\nNúmero de filas después del tratamiento: {len(df_clean)}")

# Verificar la diferencia de filas
print(f"\nFilas eliminadas: {len(df) - len(df_clean)}")